# Step 3. Train

## 1. Setup

In [2]:
! pip install -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 23.0 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scikit-learn] [scikit-learn]


In [5]:
# import necessary files

import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [6]:
# set up file paths

angles_path = "../angles_clean"

RESULTS_DIR = f"{angles_path}"
POSES = ["downdog", "goddess", "plank", "tree", "warrior2"]

# model save paths
models_path = "../models_new"

MODELS_DIR = f"{models_path}"
os.makedirs(MODELS_DIR, exist_ok=True)

MODEL_PATH = os.path.join(MODELS_DIR, "yoga_angle_resnet.pt")
META_PATH  = os.path.join(MODELS_DIR, "yoga_angle_resnet_meta.pkl")

In [7]:
# Set training parameters

batch_size = 64
num_epochs = 60
learning_rate = 1e-3
weight_decay = 1e-4
patience = 10 # for early stopping

In [9]:
df_all = pd.read_csv(f"{angles_path}/all_angles.csv")
print("all angles loaded", "shape:", df_all.shape)
display(df_all.head())
print("Columns:", list(df_all.columns))

all angles loaded shape: (945, 14)


,pose_label,image_path,left_elbow_angle,right_elbow_angle,left_shoulder_angle,right_shoulder_angle,left_knee_angle,right_knee_angle,hand_angle,left_hip_angle,right_hip_angle,neck_angle_uk,left_wrist_angle_bk,right_wrist_angle_bk
0,Downdog,../../data_clean/Downdog/Images/00000372.jpg,195.382277,189.994100,181.919382,180.159746,153.536511,156.576545,1.285724,290.903067,287.620923,247.619865,275.332105,273.613412
1,Downdog,../../data_clean/Downdog/Images/00000414.jpg,161.394529,158.980446,163.869613,198.508847,181.839432,178.438866,358.240615,83.145803,85.550792,315.000000,80.704666,81.109078
2,Downdog,../../data_clean/Downdog/Images/00000158.jpg,188.316559,189.387117,188.434129,168.625045,186.825185,185.690404,342.449527,273.259646,275.749535,335.224859,278.820494,281.517431
3,Downdog,../../data_clean/Downdog/Images/00000164.jpg,163.143669,151.368673,174.269075,190.190288,180.294451,179.480325,29.600620,79.003156,74.516612,13.561052,79.992020,75.961132
4,Downdog,../../data_clean/Downdog/Images/00000170.jpg,202.094230,196.515703,181.847610,181.956953,184.744899,186.736011,6.832362,287.592425,287.269772,205.346176,285.715038,285.775379


Columns: ['pose_label', 'image_path', 'left_elbow_angle', 'right_elbow_angle', 'left_shoulder_angle', 'right_shoulder_angle', 'left_knee_angle', 'right_knee_angle', 'hand_angle', 'left_hip_angle', 'right_hip_angle', 'neck_angle_uk', 'left_wrist_angle_bk', 'right_wrist_angle_bk']


## 2. Create datasets

In [14]:
# build X, Y
pose_col = "pose_label"
non_feature_cols = ["image_path", pose_col]

feature_cols = [c for c in df_all.columns if c not in non_feature_cols]
print("Feature columns:", feature_cols)

X = df_all[feature_cols].values.astype(np.float32)
y_names = df_all[pose_col].values

le = LabelEncoder()
y = le.fit_transform(y_names)
class_names = list(le.classes_)
print("Pose classes:", class_names)

# 70/15/15 split
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print("Train/val/test sizes:", X_train.shape[0], X_val.shape[0], X_test.shape[0])

Feature columns: ['left_elbow_angle', 'right_elbow_angle', 'left_shoulder_angle', 'right_shoulder_angle', 'left_knee_angle', 'right_knee_angle', 'hand_angle', 'left_hip_angle', 'right_hip_angle', 'neck_angle_uk', 'left_wrist_angle_bk', 'right_wrist_angle_bk']
Pose classes: ['Downdog', 'Goddess', 'Plank', 'Tree', 'Warrior2']
Train/val/test sizes: 686 147 148


In [15]:
# dataset
class AngleDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.int64)
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self, idx):
        x = torch.from_numpy(self.X[idx])
        y = torch.tensor(self.y[idx], dtype=torch.long)
        return x, y

train_ds = AngleDataset(X_train, y_train)
val_ds = AngleDataset(X_val, y_val)
test_ds = AngleDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

## 3. Train MLP

TODO: residual MLP
different parameters?

In [ ]:
# residual MLP model
class ResidualBlock(nn.Module):
    def __init__(self, dim, dropout=0.5):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim),
        )
        self.act = nn.ReLU()

    def forward(self, x):
        out = self.net(x)
        out = out + x  # skip connection
        out = self.act(out)
        return out


class AngleResNet(nn.Module):
    def __init__(self, in_dim, num_classes, hidden_dim=256, num_blocks=3, dropout=0.5):
        super().__init__()
        self.input_layer = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
        )
        self.blocks = nn.Sequential(*[
            ResidualBlock(hidden_dim, dropout=dropout)
            for _ in range(num_blocks)
        ])
        self.head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, num_classes)
        )

    def forward(self, x):
        h = self.input_layer(x)
        h = self.blocks(h)
        logits = self.head(h)
        return logits

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

in_dim = X.shape[1]
num_classes = len(class_names)
model = AngleResNet(in_dim, num_classes, hidden_dim=256, num_blocks=3, dropout=0.5).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=weight_decay
)

Using device: cuda


In [17]:
# train loop
def run_epoch(loader, model, criterion, optimizer=None, device="cpu"):
    if optimizer is None:
        model.eval()
    else:
        model.train()

    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    all_preds = []
    all_targets = []

    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        if optimizer is not None:
            optimizer.zero_grad()

        logits = model(X_batch)
        loss = criterion(logits, y_batch)

        if optimizer is not None:
            loss.backward()
            optimizer.step()

        total_loss += float(loss.item()) * X_batch.size(0)
        preds = logits.argmax(dim=1)
        total_correct += (preds == y_batch).sum().item()
        total_samples += X_batch.size(0)

        all_preds.append(preds.detach().cpu().numpy())
        all_targets.append(y_batch.detach().cpu().numpy())

    avg_loss = total_loss / total_samples
    acc = total_correct / total_samples
    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)

    return avg_loss, acc, all_preds, all_targets

In [18]:
# training loop
best_val_acc = 0.0
best_state = None
epochs_no_improve = 0

for epoch in range(1, num_epochs + 1):
    train_loss, train_acc, _, _ = run_epoch(
        train_loader, model, criterion, optimizer, device=device
    )
    val_loss, val_acc, _, _ = run_epoch(
        val_loader, model, criterion, optimizer=None, device=device
    )

    print(f"Epoch {epoch:02d} | "
          f"train_loss={train_loss:.3f} acc={train_acc:.3f} | "
          f"val_loss={val_loss:.3f} acc={val_acc:.3f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_state = model.state_dict()
        epochs_no_improve = 0
        print(f"  ↳ New best model (val_acc={best_val_acc:.3f})")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f"[EARLY STOP] No val improvement for {patience} epochs.")
            break

if best_state is not None:
    model.load_state_dict(best_state)

Epoch 01 | train_loss=1.254 acc=0.551 | val_loss=1.205 acc=0.517
  ↳ New best model (val_acc=0.517)
Epoch 02 | train_loss=0.630 acc=0.800 | val_loss=0.415 acc=0.884
  ↳ New best model (val_acc=0.884)
Epoch 03 | train_loss=0.320 acc=0.901 | val_loss=0.214 acc=0.946
  ↳ New best model (val_acc=0.946)
Epoch 04 | train_loss=0.254 acc=0.931 | val_loss=0.169 acc=0.952
  ↳ New best model (val_acc=0.952)
Epoch 05 | train_loss=0.168 acc=0.948 | val_loss=0.155 acc=0.959
  ↳ New best model (val_acc=0.959)
Epoch 06 | train_loss=0.166 acc=0.945 | val_loss=0.160 acc=0.952
Epoch 07 | train_loss=0.110 acc=0.962 | val_loss=0.159 acc=0.959
Epoch 08 | train_loss=0.103 acc=0.971 | val_loss=0.158 acc=0.966
  ↳ New best model (val_acc=0.966)
Epoch 09 | train_loss=0.086 acc=0.977 | val_loss=0.171 acc=0.966
Epoch 10 | train_loss=0.108 acc=0.971 | val_loss=0.166 acc=0.973
  ↳ New best model (val_acc=0.973)
Epoch 11 | train_loss=0.108 acc=0.969 | val_loss=0.164 acc=0.973
Epoch 12 | train_loss=0.096 acc=0.975 | 

In [19]:
# evaluate on test set
test_loss, test_acc, preds_test, targets_test = run_epoch(
    test_loader, model, criterion, optimizer=None, device=device
)
print(f"Test loss={test_loss:.3f} acc={test_acc:.3f}")

print("\nClassification report:")
print(classification_report(targets_test, preds_test, target_names=class_names))

Test loss=0.123 acc=0.980

Classification report:
              precision    recall  f1-score   support

     Downdog       1.00      1.00      1.00        29
     Goddess       0.97      0.93      0.95        30
       Plank       0.97      1.00      0.98        29
        Tree       1.00      1.00      1.00        30
    Warrior2       0.97      0.97      0.97        30

    accuracy                           0.98       148
   macro avg       0.98      0.98      0.98       148
weighted avg       0.98      0.98      0.98       148



In [20]:
# save model
import pickle

# Per-pose angle stats using full dataset X, y
angle_stats = {}
for cls_idx, pose_name in enumerate(class_names):
    mask = (y == cls_idx)
    X_pose = X[mask]
    mean_angles = X_pose.mean(axis=0)
    std_angles  = X_pose.std(axis=0)
    angle_stats[pose_name] = {
        "mean": mean_angles,
        "std": std_angles,
    }

# save model
torch.save(model.state_dict(), MODEL_PATH)
print("Saved model to:", MODEL_PATH)

# save meta for inference/feedback
meta = {
    "class_names": class_names,
    "feature_cols": feature_cols,
    "angle_stats": angle_stats,
    "hidden_dim": 256,
}
with open(META_PATH, "wb") as f:
    pickle.dump(meta, f)

print("Saved meta to:", META_PATH)

Saved model to: models/yoga_angle_resnet.pt
Saved meta to: models/yoga_angle_resnet_meta.pkl
